# Linear Algebra in OSCAR

* Authors: Martin Bies, Aaruni Kaushik
* Version: OSCAR version 1.2.2 or newer.

This tutorial provides an introduction to linear algebra in OSCAR. The aim is to describe important functionality and to exemplify it, so that users can start to use OSCAR more easily. Still, let us note that this tutorial does not aim to cover all of the existing functionality. Instead, we present a selection of arguably very important functions and features. For a more complete overview, please consult the documentation at https://docs.oscar-system.org/stable/LinearAlgebra/intro/.

For this tutorial, the reader need not be familiar with OSCAR, but is assumed to have some basic understanding of Julia.

The content of this notebook is as follows:
1. [Creating Matrices](#1-Creating-Matrices)
2. [Solving Linear Equation Systems](#2-Solving-Linear-Equation-Systems)
3. [Eigenvalues and Eigenspaces](#3-Eigenvalues-and-Eigenspaces)
4. [Normalforms](#4-Normalforms)


In [1]:
using Oscar
using Test

In [2]:
Oscar.versioninfo()

OSCAR version 1.2.2
  combining:
    AbstractAlgebra.jl   v0.44.2
    GAP.jl               v0.12.3
    Hecke.jl             v0.35.7
    Nemo.jl              v0.48.0
    Polymake.jl          v0.11.24
    Singular.jl          v0.24.1


## 1 Creating Matrices

Crucially, OSCAR its own matrix type. Details of the latter are available here: https://docs.oscar-system.org/stable/AbstractAlgebra/matrix/#Matrix-functionality

The central reason is that every matrix should have a unique coefficient ring assigned and for a Julia matrix this is not possible. Types (even with parameters) are not suitable for this purpose. For details, see (link to be inserted).


In the following, we will elaborate on how we create such OSCAR matrices. For conversion of the "normal" julia matrices into OSCAR matrices, see for instance https://docs.oscar-system.org/stable/AbstractAlgebra/matrix/#Conversion-to-Julia-matrices,-iteration-and-broacasting.

There are many ways to create matrices in $\texttt{OSCAR}$. We demonstrate a few.

In constrast to ordinary Julia matrices, please note that we specify the specify the coefficients ring.

In [3]:
m = matrix(ZZ, [1 2 3 ; 4 5 6])

[1   2   3]
[4   5   6]

We can create a matrix with the same coefficients over the rationals:

In [4]:
m2 = matrix(QQ, [1 2 3; 4 5 6])

[1   2   3]
[4   5   6]

Note that these two matrices print the same but they have different types.

In [5]:
typeof(m)

ZZMatrix

In [6]:
typeof(m2)

QQMatrix

Despite these matrices being defined over different coefficient rings, we can (for instance) add them:

In [7]:
m+m2

[2    4    6]
[8   10   12]

Likewise a comparison is possible:

In [8]:
m == m2

true

This fails for other coefficient rings. As an example, let us consider the following matrices:

In [9]:
F, a = finite_field(9, "a")
If = identity_matrix(F, 4)

[1   0   0   0]
[0   1   0   0]
[0   0   1   0]
[0   0   0   1]

In [10]:
Iz = identity_matrix(ZZ, 4)

[1   0   0   0]
[0   1   0   0]
[0   0   1   0]
[0   0   0   1]

In [11]:
Iq = identity_matrix(QQ, 4)

[1   0   0   0]
[0   1   0   0]
[0   0   1   0]
[0   0   0   1]

Note that we used a convenience functions to create these identity matrices. More such functionality does exist, see e.g. https://docs.oscar-system.org/stable/AbstractAlgebra/matrix/#Basic-matrix-functionality. Here is another example use of such a convenience function:

In [12]:
zero_matrix(ZZ, 4, 5)

[0   0   0   0   0]
[0   0   0   0   0]
[0   0   0   0   0]
[0   0   0   0   0]

Let us now return to our matrices $Iz$, $Iq$ and $If$. Those are defined over different coefficient rings. While they print the same, not all pairs of them can be compared:

In [13]:
Iz == Iq

true

In [14]:
Iz == If

true

In [15]:
@test_broken If == Iq

Test Broken
  Expression: If == Iq

This is not a bug! Rather, in general, elements of different rings cannot be compared (or added). In other words: When matrices are defined over different coefficient rings, then in general they behave differently and cannot be added, compared, etc.

## 2 Solving Linear Equation Systems

Let us solve the following system of linear equations in $\texttt{OSCAR}$ over $\mathbb{Q}$:

$$ \left[ \begin{array}{ccc} 1 & 2 & 3 \\ 2 & 4 & 6 \end{array} \right] \cdot v = \left[ \begin{array}{c} -5 \\ -10 \end{array} \right] $$

In [16]:
m = matrix(QQ, [[1,2,3], [2,4,6]])

[1   2   3]
[2   4   6]

In [17]:
b = matrix(QQ, [[-5], [-10]])

[ -5]
[-10]

In [18]:
solve(m, b, side = :right)

[-5]
[ 0]
[ 0]

But of course, this is merely a single solution. To obtain all solutions, we need to add the nullspace.

In [19]:
nullspace(m)

(2, [-2 -3; 1 0; 0 1])

The first return value is the dimension of the nullspace, and the second is a basis of the nullspace.

In [20]:
nullspace(m)[2]

[-2   -3]
[ 1    0]
[ 0    1]

Additional information is available in the OSCAR documentation. See for instance:
* https://docs.oscar-system.org/stable/AbstractAlgebra/linear_solving/#solving_chapter.
* https://docs.oscar-system.org/stable/AbstractAlgebra/matrix/#nullspace-Union{Tuple{MatElem{T}},%20Tuple{T}}%20where%20T%3C:FieldElem

## 3 Eigenvalues and Eigenspaces

In this section, we demonstrate how to find eigenvalues, and eigenspaces of a diagonalizable matrix.
For simplicity, we focus on the case of a matrix that is diagonalizable over $\mathbb{Q}$

In [21]:
M = matrix(QQ, [[-16, -28, -19], [42, 69, 46], [-42, -72, -49]])

[-16   -28   -19]
[ 42    69    46]
[-42   -72   -49]

In [22]:
det(M)

-30

In [23]:
characteristic_polynomial(M)

x^3 - 4*x^2 - 11*x + 30

In [24]:
eigenvalues(QQBarField(), M)

3-element Vector{QQBarFieldElem}:
 Root 5.00000 of x - 5
 Root 2.00000 of x - 2
 Root -3.00000 of x + 3

In [25]:
A = vcat([eigenspace(M, QQ(5)), eigenspace(M, QQ(2)), eigenspace(M, QQ(-3))]...)

[2   2   1]
[7   4   1]
[0   1   1]

In [26]:
D = diagonal_matrix(QQ, [5, 2, -3])

[5   0    0]
[0   2    0]
[0   0   -3]

In [27]:
(inv(A) * D * A) == M

true

For more details see e.g. https://docs.oscar-system.org/stable/Nemo/algebraic/#eigenvalues-Tuple{QQBarField,%20ZZMatrix}.

## 4 Normalforms

There are many many normal forms, we demonstrate how to compute the Smith and the Hermite normal form.

### 4.1 Smith Normal Form

In [28]:
M

[-16   -28   -19]
[ 42    69    46]
[-42   -72   -49]

In [29]:
snf(M)

[1   0   0]
[0   1   0]
[0   0   1]

In [30]:
S, T, U = snf_with_transform(M)

([1 0 0; 0 1 0; 0 0 1], [1 0 0; 21//80 -7//240 -31//240; 21//5 -4//5 -12//5], [-1//16 28 -19//16; 0 -16 0; 0 0 1])

In [31]:
M == inv(T) * S * inv(U)

true

 For more details see https://docs.oscar-system.org/stable/AbstractAlgebra/matrix/#Smith-normal-form.

### 4.2 Hermite Normal Form

In [32]:
M

[-16   -28   -19]
[ 42    69    46]
[-42   -72   -49]

In [33]:
hnf(M)

[1   0   0]
[0   1   0]
[0   0   1]

In [34]:
H, U = hnf_with_transform(M)

([1 0 0; 0 1 0; 0 0 1], [23//10 2//15 -23//30; -21//5 7//15 31//15; 21//5 -4//5 -12//5])

In [35]:
M == inv(U) * H

true

For more details see https://docs.oscar-system.org/stable/AbstractAlgebra/matrix/#Hermite-normal-form.

## 5 Vector

One way of creating vectors, is by extracting rows and columns of a matrix. Let us demonstrate this:

In [36]:
M = matrix(QQ, [[1,2,3], [4,5,6]])

[1   2   3]
[4   5   6]

In [37]:
row = M[1:1,:]

[1   2   3]

In [38]:
col = M[:,1:1]

[1]
[4]

Certainly, we can multiply the original matrix from the right with the transpose of row:

In [39]:
M * transpose(row)

[14]
[32]

Likewise, we can multiply the original matrix from the left with the transpose of col:

In [40]:
transpose(col) * M

[17   22   27]

Incompatible matrix dimensions lead to errors. In the case at hand, this is for instance the case here:

In [41]:
@test_broken M * col

Test Broken
  Expression: M * col

In [42]:
@test_broken col * M

Test Broken
  Expression: col * M